In [1]:
! pip install keras
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from tqdm import tqdm
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm.keras import TqdmCallback


# 1 Hour

In [5]:
# We take the document when we have all our data normalized:
df_1H = pd.read_csv('/content/sample_data/Full_data_1H.csv')

In [8]:
df_1H.head()

,Date_1H,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,collapse,ATH,grow,rise,fall,date,open,close,Return,Return_sign
0,2022-01-01 01:00:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.755714,0.511428,0.364813,0.161558,34,72,...,2,28,10,20,65,2022-01-01 01:00:00,46656.14,46778.14,0.002615,1
1,2022-01-01 02:00:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.775810,0.551621,0.343045,0.174069,30,29,...,0,14,13,13,30,2022-01-01 02:00:00,46778.14,46811.77,0.000719,1
2,2022-01-01 03:00:00,"['2022-01-01 02:00:00', '2022-01-01 02:00:00',...",Huobi Singapore would like to wish you a Happy...,HuobiSg\nAmazonCash4Cars\nBTC_AI_bot\nCalvinAy...,0.773046,0.546093,0.427992,0.181803,23,24,...,1,20,10,11,24,2022-01-01 03:00:00,46811.77,46813.20,0.000031,1
3,2022-01-01 04:00:00,"['2022-01-01 03:00:00', '2022-01-01 03:00:00',...",1 ARS is 10.26 satoshis right now.\n\n1 satosh...,SatoshiARS_bot\nbitcoinpartyall\ncryptopediaK7...,0.760779,0.521558,0.325186,0.175078,26,20,...,1,24,13,12,22,2022-01-01 04:00:00,46813.21,46711.05,-0.002182,0
4,2022-01-01 05:00:00,"['2022-01-01 04:00:00', '2022-01-01 04:00:00',...",Buy Bitcoin $BTC @ 46813.2\nAn online trader h...,BTC_AI_bot\nnewscomauHQ\ncryptozone007\nCarbon...,0.771374,0.542747,0.377858,0.152699,26,19,...,1,22,7,12,16,2022-01-01 05:00:00,46711.05,47192.55,0.010308,1


In [7]:
df_1H.columns

Index(['Date_1H', 'datetime', 'text', 'username', 'kk08_pos_score',
       'kk08_score_diff', 'Elkulako_label', 'Sentiment_vader', 'pred',
       'analysis', 'short', 'long', 'buy', 'sell', 'bull', 'bear', 'bankrupt',
       'fraud', 'cbdc', 'record', 'crisis', 'collapse', 'ATH', 'grow', 'rise',
       'fall', 'date', 'open', 'close', 'Return', 'Return_sign'],
      dtype='object')

In [12]:
# I just colect the information i will need to use to train the NN for each of the sentiment results:
df_1H_Vader = df_1H[['Sentiment_vader','Return_sign']]
df_1H_kk08 = df_1H[['kk08_pos_score','Return_sign']]
df_1H_Elkulako = df_1H[['Elkulako_label','Return_sign']]


We define the model we want to use

In [14]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(1,)),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Define callbacks to stop and dont overfit the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

## KK08

In [17]:
df_1H_kk08.columns

Index(['kk08_pos_score', 'Return_sign'], dtype='object')

In [24]:
X_kk08 = df_1H_kk08 ['kk08_pos_score']
Y_kk08 = df_1H_kk08 ['Return_sign']
#Y_kk08 = np.array(df_1H_kk08 ['Return_sign'])

In [23]:
train_share = 0.85
df_x_1_train = X_kk08.iloc[:int(len(X_kk08)*train_share)].reset_index(drop=True)
df_y_train = Y_kk08.iloc[:int(len(Y_kk08)*train_share)].reset_index(drop=True)

df_x_1_test = X_kk08.iloc[int(len(X_kk08)*train_share):].reset_index(drop=True)
df_y_test = Y_kk08.iloc[int(len(X_kk08)*train_share):].reset_index(drop=True)

In [78]:
df_x_1_train.shape

(632,)

In [25]:
# Checking that class distribution is more or less balance in both dataset.
print("Training set class distribution:")
print(pd.Series(df_y_train).value_counts())
print("Test set class distribution:")
print(pd.Series(df_y_test).value_counts())

Training set class distribution:
Return_sign
1    322
0    310
Name: count, dtype: int64
Test set class distribution:
Return_sign
1    59
0    53
Name: count, dtype: int64


In [29]:
train_share = 0.90
split_index = int(train_share * len(df_x_1_train))
X_train, X_valid = df_x_1_train[:split_index], df_x_1_train[split_index:]
y_train, y_valid = df_y_train[:split_index], df_y_train[split_index:]

In [30]:
history = model.fit(
    df_x_1_train, df_y_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_valid,  y_valid),
    callbacks=[early_stopping, model_checkpoint, TqdmCallback(verbose=1)],
    verbose=0
)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [32]:
# Evaluar en el conjunto de validación o de prueba
test_loss, test_accuracy = model.evaluate(df_x_1_test, df_y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4847 - loss: 0.6937  
Test Loss: 0.6924704909324646
Test Accuracy: 0.5267857313156128


In [35]:
from sklearn.metrics import roc_auc_score

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(df_x_1_test, df_y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predict probabilities for the positive class
y_pred_probs = model.predict(df_x_1_test)

# Use df_y_test directly as y_true if it's already binary (0 or 1)
y_true = df_y_test  # Assuming df_y_test is not one-hot encoded

# Compute the ROC AUC score
roc_auc = roc_auc_score(y_true, y_pred_probs)
print(f"ROC AUC: {roc_auc}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4847 - loss: 0.6937 
Test Loss: 0.6924704909324646
Test Accuracy: 0.5267857313156128
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
ROC AUC: 0.5062360089542692


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

In [37]:
# Since the first result was not completely satisfactory, we try to improve the solution proving adding or remove layers of a NN to se how we can improve the NN:
def create_model(num_layers=6, neurons_per_layer=48, input_shape=(10,), output_units=1, activation='relu'):
    model = Sequential()
    model.add(Dense(neurons_per_layer, activation=activation, input_shape=input_shape))

    # Agregar las capas ocultas especificadas
    for _ in range(num_layers - 1):
        model.add(Dense(neurons_per_layer, activation=activation))

    # Capa de salida
    model.add(Dense(output_units, activation='sigmoid'))  # Cambia 'sigmoid' a 'softmax' si tienes una clasificación multiclase

    # Compilar el modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model




# Definir parámetros de la red
num_layers_list = [2, 3, 4,6,8]  # Diferentes números de capas a probar
neurons_per_layer_list = [16, 32, 64,128]  # Diferentes números de neuronas en cada capa

# Probar diferentes configuraciones de capas y neuronas
for num_layers in num_layers_list:
    for neurons_per_layer in neurons_per_layer_list:
        print(f"\nModel trained with {num_layers} hidden layers andy {neurons_per_layer} neurons per layer")

        # Crear el modelo
        model = create_model(num_layers=num_layers, neurons_per_layer=neurons_per_layer, input_shape=(1,))

        # Definir early stopping para evitar el sobreajuste
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Entrenar el modelo
        history = model.fit(df_x_1_train, df_y_train, validation_split=0.2, epochs=200, batch_size=16, callbacks=[early_stopping,TqdmCallback(verbose=1)], verbose=1)

        # Evaluar el modelo
        loss, accuracy = model.evaluate(df_x_1_test, df_y_test, verbose=0)
        print(f"Model with {num_layers} layers y {neurons_per_layer} neurons -> Loss: {loss:.4f}, accuracy: {accuracy:.4f}")

        # Predict probabilities for the positive class
        y_pred_probs = model.predict(df_x_1_test)

        # Use df_y_test directly as y_true if it's already binary (0 or 1)
        y_true = df_y_test  # Assuming df_y_test is not one-hot encoded

        # Compute the ROC AUC score
        roc_auc = roc_auc_score(y_true, y_pred_probs)
        print(f"ROC AUC: {roc_auc}")




Entrenando modelo con 2 capas ocultas y 16 neuronas por capa


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.5135 - loss: 0.6932 - val_accuracy: 0.4409 - val_loss: 0.6969
Epoch 2/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4944 - loss: 0.6938 - val_accuracy: 0.4409 - val_loss: 0.6943
Epoch 3/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4881 - loss: 0.6937 - val_accuracy: 0.4409 - val_loss: 0.6936
Epoch 4/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5159 - loss: 0.6929 - val_accuracy: 0.4409 - val_loss: 0.6955
Epoch 5/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4632 - loss: 0.6950 - val_accuracy: 0.4409 - val_loss: 0.6938
Epoch 6/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5075 - loss: 0.6932 - val_accuracy: 0.4409 - val_loss: 0.6947
Epoch 7/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5016 - loss: 0.6933 - val_accuracy: 0.4409 - val_loss: 0.6941
Epoch 8/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4967 - loss: 0.6933 - val_accuracy: 0.4409 

...

Epoch 1/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.4838 - loss: 0.6936 - val_accuracy: 0.4409 - val_loss: 0.6934
Epoch 2/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4824 - loss: 0.6934 - val_accuracy: 0.4409 - val_loss: 0.6937
Epoch 3/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5235 - loss: 0.6931 - val_accuracy: 0.4409 - val_loss: 0.6937
Epoch 4/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4838 - loss: 0.6932 - val_accuracy: 0.4409 - val_loss: 0.6932
Epoch 5/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5100 - loss: 0.6932 - val_accuracy: 0.4409 - val_loss: 0.6937
Epoch 6/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5200 - loss: 0.6931 - val_accuracy: 0.4409 - val_loss: 0.6938
Epoch 7/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4944 - loss: 0.6932 - val_accuracy: 0.5591 - val_loss: 0.6931
Epoch 8/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4923 - loss: 0.6932 - val_accuracy: 0.4409 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.4514 - loss: 0.6944 - val_accuracy: 0.5591 - val_loss: 0.6931
Epoch 2/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5340 - loss: 0.6934 - val_accuracy: 0.4409 - val_loss: 0.6937
Epoch 3/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5007 - loss: 0.6933 - val_accuracy: 0.4409 - val_loss: 0.6945
Epoch 4/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5106 - loss: 0.6931 - val_accuracy: 0.4409 - val_loss: 0.6941
Epoch 5/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4326 - loss: 0.6938 - val_accuracy: 0.4409 - val_loss: 0.6933
Epoch 6/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5079 - loss: 0.6932 - val_accuracy: 0.4409 - val_loss: 0.6942
Model with 8 layers y 128 neurons -> Loss: 0.6931, accuracy: 0.5268
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
ROC AUC: 0.49424368404221297


The best results were given when : Model with 8 layers y 64 neurons ->

*Loss: 0.6931,

*accuracy: 0.5268

*ROC AUC: 0.49600255836264784

## With Lag

In [38]:
df_1H['kk08_pos_score_lag1']=df_1H['kk08_pos_score'].shift(1)
df_1H['kk08_pos_score_lag2']=df_1H['kk08_pos_score'].shift(2)
df_1H['kk08_pos_score_lag3']=df_1H['kk08_pos_score'].shift(3)

In [45]:
df_x_2 = df_1H.dropna().reset_index(drop=True)[["kk08_pos_score", "kk08_pos_score_lag1", "kk08_pos_score_lag2", "kk08_pos_score_lag3"]]
df_y_2 = df_1H["Return_sign"].iloc[3:].reset_index(drop=True)

In [70]:
df_y_2.shape


(741,)

In [95]:
train_share = 0.85
df_x_2_train = df_x_2.iloc[:int(len(df_x_2)*train_share)].reset_index(drop=True)
df_y_2_train = df_y_2.iloc[:int(len(df_x_2)*train_share)].reset_index(drop=True)

df_x_2_test = df_x_2.iloc[int(len(df_x_2)*train_share):].reset_index(drop=True)
df_y_2_test = df_y_2.iloc[int(len(df_x_2)*train_share):].reset_index(drop=True)

In [96]:
train_share = 0.90
split_index = int(train_share * len(df_x_1_train))
X_2train, X_2valid = df_x_2_train[:split_index], df_x_2_train[split_index:]
y_2train, y_2valid = df_y_2_train[:split_index], df_y_2_train[split_index:]

In [64]:
X = df_x_2.fillna(0).to_numpy()  # Fills NaNs with 0 (or any other value)
# Or drop rows with NaN values:
X = df_x_2.dropna().to_numpy()

In [85]:
X.shape

(629, 4)

In [87]:
model_lag = Sequential([
    Dense(32, activation='relu', input_shape=(4,)),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [92]:
print("Shape of df_x_2_train:", df_x_2_train.shape)
print("Shape of df_y_2_train:", df_y_2_train.shape)

Shape of df_x_2_train: (629, 4)
Shape of df_y_2_train: (629,)


In [97]:
print("Shape of X_2valid:", X_2valid.shape)
print("Shape of y_2valid:", y_2valid.shape)


Shape of X_2valid: (61, 4)
Shape of y_2valid: (61,)


In [98]:
model_lag.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [99]:
history = model_lag.fit(
    df_x_2_train, df_y_2_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_2valid, y_2valid),
    callbacks=[early_stopping, model_checkpoint, TqdmCallback(verbose=1)],
    verbose=0
)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

# Merge feelings and returns from Elkulako (Ignore)

In [17]:
df_elkulako = pd.read_csv('/content/sample_data/Scores_ElKulako (1).csv')

In [18]:
df_elkulako

,0
0,"{'label': 'Neutral', 'score': 0.7601109743118286}"
1,"{'label': 'Neutral', 'score': 0.7919839024543762}"
2,"{'label': 'Neutral', 'score': 0.5934157371520996}"
3,"{'label': 'Bullish', 'score': 0.5577491521835327}"
4,"{'label': 'Neutral', 'score': 0.6853246688842773}"
...,...
1331663,"{'label': 'Bullish', 'score': 0.591331958770752}"
1331664,"{'label': 'Neutral', 'score': 0.6384872794151306}"
1331665,"{'label': 'Neutral', 'score': 0.8317734003067017}"
1331666,"{'label': 'Neutral', 'score': 0.9091027975082397}"


In [19]:
import ast


In [20]:
# Step 1: Convert the JSON-like strings into dictionaries
df_elkulako['0'] = df_elkulako['0'].apply(ast.literal_eval)

# Step 2: Extract 'label' and 'score' into new columns
df_elkulako['label'] = df_elkulako['0'].apply(lambda x: x['label'])
df_elkulako['score'] = df_elkulako['0'].apply(lambda x: x['score'])

In [21]:
df_elkulako


,0,label,score
0,"{'label': 'Neutral', 'score': 0.7601109743118286}",Neutral,0.760111
1,"{'label': 'Neutral', 'score': 0.7919839024543762}",Neutral,0.791984
2,"{'label': 'Neutral', 'score': 0.5934157371520996}",Neutral,0.593416
3,"{'label': 'Bullish', 'score': 0.5577491521835327}",Bullish,0.557749
4,"{'label': 'Neutral', 'score': 0.6853246688842773}",Neutral,0.685325
...,...,...,...
1331663,"{'label': 'Bullish', 'score': 0.591331958770752}",Bullish,0.591332
1331664,"{'label': 'Neutral', 'score': 0.6384872794151306}",Neutral,0.638487
1331665,"{'label': 'Neutral', 'score': 0.8317734003067017}",Neutral,0.831773
1331666,"{'label': 'Neutral', 'score': 0.9091027975082397}",Neutral,0.909103


In [22]:
df_elkulako['Positive_score'] = df_elkulako.apply(lambda row: 1 - row['score'] if row['label'] == 'Bearish' else row['score'], axis=1)


In [23]:
#Type of labels in column labels :
df_elkulako['label'].unique()

array(['Neutral', 'Bullish', 'Bearish'], dtype=object)

In [24]:
df_elkulako

,0,label,score,Positive_score
0,"{'label': 'Neutral', 'score': 0.7601109743118286}",Neutral,0.760111,0.760111
1,"{'label': 'Neutral', 'score': 0.7919839024543762}",Neutral,0.791984,0.791984
2,"{'label': 'Neutral', 'score': 0.5934157371520996}",Neutral,0.593416,0.593416
3,"{'label': 'Bullish', 'score': 0.5577491521835327}",Bullish,0.557749,0.557749
4,"{'label': 'Neutral', 'score': 0.6853246688842773}",Neutral,0.685325,0.685325
...,...,...,...,...
1331663,"{'label': 'Bullish', 'score': 0.591331958770752}",Bullish,0.591332,0.591332
1331664,"{'label': 'Neutral', 'score': 0.6384872794151306}",Neutral,0.638487,0.638487
1331665,"{'label': 'Neutral', 'score': 0.8317734003067017}",Neutral,0.831773,0.831773
1331666,"{'label': 'Neutral', 'score': 0.9091027975082397}",Neutral,0.909103,0.909103


In [25]:
df_elkulako[df_elkulako['label']=="Bearish"]

,0,label,score,Positive_score
6,"{'label': 'Bearish', 'score': 0.6867599487304688}",Bearish,0.686760,0.313240
15,"{'label': 'Bearish', 'score': 0.7277977466583252}",Bearish,0.727798,0.272202
20,"{'label': 'Bearish', 'score': 0.9384021759033203}",Bearish,0.938402,0.061598
113,"{'label': 'Bearish', 'score': 0.82611483335495}",Bearish,0.826115,0.173885
149,"{'label': 'Bearish', 'score': 0.9663400053977966}",Bearish,0.966340,0.033660
...,...,...,...,...
1331562,"{'label': 'Bearish', 'score': 0.6561643481254578}",Bearish,0.656164,0.343836
1331577,"{'label': 'Bearish', 'score': 0.9632375836372375}",Bearish,0.963238,0.036762
1331623,"{'label': 'Bearish', 'score': 0.5134078860282898}",Bearish,0.513408,0.486592
1331656,"{'label': 'Bearish', 'score': 0.5800412893295288}",Bearish,0.580041,0.419959


In [26]:
df_clean_tweets = pd.read_csv('/content/sample_data/Cleaned_tweets (1).csv')

In [44]:
current_df = pd.concat([df_clean_tweets,df_elkulako["Positive_score"]], axis=1)

In [45]:
current_df

,datetime,date,username,text,Date_hourly,Date_min,Date_day,Date_30m,Date_15m,cleaned_text,Positive_score
0,2022-01-01 00:00:00+00:00,2022-01-01,CryptoNerdApp,Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,2022-01-01 01:00:00+00:00,2022-01-01 00:01:00+00:00,2022-01-02 00:00:00+00:00,2022-01-01 00:30:00+00:00,2022-01-01 00:15:00+00:00,current price bitcoin btc bitcoin cryptocurren...,0.760111
1,2022-01-01 17:10:02+00:00,2022-01-01,bitcoinalerts,The year for Bitcoin — A 2021 roundup of the f...,2022-01-01 18:00:00+00:00,2022-01-01 17:11:00+00:00,2022-01-02 00:00:00+00:00,2022-01-01 17:30:00+00:00,2022-01-01 17:15:00+00:00,year bitcoin roundup flagship crypto,0.791984
2,2022-01-01 17:10:00+00:00,2022-01-01,oneandonlypd,#Bitcoin is all our savings. https://t.co/SGgy...,2022-01-01 18:00:00+00:00,2022-01-01 17:11:00+00:00,2022-01-02 00:00:00+00:00,2022-01-01 17:30:00+00:00,2022-01-01 17:15:00+00:00,bitcoin saving,0.593416
3,2022-01-01 17:09:59+00:00,2022-01-01,CHAIRFORCE_BTC,@gmekhail Isn't that usually shouted at the pe...,2022-01-01 18:00:00+00:00,2022-01-01 17:10:00+00:00,2022-01-02 00:00:00+00:00,2022-01-01 17:30:00+00:00,2022-01-01 17:15:00+00:00,isnt usually shouted people promoting going bi...,0.557749
4,2022-01-01 17:09:58+00:00,2022-01-01,crypto_squared,What are Decentralized Apps or DApps?\n#Crypto...,2022-01-01 18:00:00+00:00,2022-01-01 17:10:00+00:00,2022-01-02 00:00:00+00:00,2022-01-01 17:30:00+00:00,2022-01-01 17:15:00+00:00,decentralized apps dapps cryptonews bitcoin cr...,0.685325
...,...,...,...,...,...,...,...,...,...,...,...
1331663,2022-01-31 10:17:58+00:00,2022-01-31,cstross,@ignaziop1977 @MLK3030 @SHOKUNIN_STUDIO At the...,2022-01-31 11:00:00+00:00,2022-01-31 10:18:00+00:00,2022-02-01 00:00:00+00:00,2022-01-31 10:30:00+00:00,2022-01-31 10:30:00+00:00,point wrote book unaware existence blockchainb...,0.591332
1331664,2022-01-31 10:18:02+00:00,2022-01-31,S_Edalati,Cryptocurrency Prices On January 31 2021: Know...,2022-01-31 11:00:00+00:00,2022-01-31 10:19:00+00:00,2022-02-01 00:00:00+00:00,2022-01-31 10:30:00+00:00,2022-01-31 10:30:00+00:00,cryptocurrency price january know rate bitcoin...,0.638487
1331665,2022-01-31 10:18:05+00:00,2022-01-31,bitcoinpressuk,#Bitcoin #BTC #CRYPTO US Financial Advisors Ex...,2022-01-31 11:00:00+00:00,2022-01-31 10:19:00+00:00,2022-02-01 00:00:00+00:00,2022-01-31 10:30:00+00:00,2022-01-31 10:30:00+00:00,bitcoin btc crypto us financial advisor expect...,0.831773
1331666,2022-01-31 10:17:28+00:00,2022-01-31,AlfaHedge,How low can #Bitcoin go ???\n\nExperts weigh i...,2022-01-31 11:00:00+00:00,2022-01-31 10:18:00+00:00,2022-02-01 00:00:00+00:00,2022-01-31 10:30:00+00:00,2022-01-31 10:30:00+00:00,low bitcoin go expert weigh cryptonews cryptoc...,0.909103


In [46]:
current_df = current_df.sort_values(by="datetime").reset_index(drop=True)

In [47]:
current_df['datetime'] = pd.to_datetime(current_df['datetime']).dt.tz_localize(None)
current_df['Date_hourly'] = pd.to_datetime(current_df['Date_hourly']).dt.tz_localize(None)
current_df['Date_min'] = pd.to_datetime(current_df['Date_min']).dt.tz_localize(None)
current_df['Date_15m'] = pd.to_datetime(current_df['Date_15m']).dt.tz_localize(None)
current_df['Date_30m'] = pd.to_datetime(current_df['Date_30m']).dt.tz_localize(None)

In [31]:
current_df

,datetime,date,username,text,Date_hourly,Date_min,Date_day,Date_30m,Date_15m,cleaned_text,Positive_score
0,2022-01-01 00:00:00,2022-01-01,CryptoNerdApp,Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,2022-01-01 01:00:00,2022-01-01 00:01:00,2022-01-02 00:00:00+00:00,2022-01-01 00:30:00,2022-01-01 00:15:00,current price bitcoin btc bitcoin cryptocurren...,0.760111
1,2022-01-01 00:00:00,2022-01-01,TrendSpider,$BTC Continues to bounce off the YTD anchored ...,2022-01-01 01:00:00,2022-01-01 00:01:00,2022-01-02 00:00:00+00:00,2022-01-01 00:30:00,2022-01-01 00:15:00,btc continues bounce ytd anchored vwap bitcoin,0.923622
2,2022-01-01 00:00:00,2022-01-01,MadStudentScie1,Generating misunderstanding for Bitcoin!,2022-01-01 01:00:00,2022-01-01 00:01:00,2022-01-02 00:00:00+00:00,2022-01-01 00:30:00,2022-01-01 00:15:00,generating misunderstanding bitcoin,0.252997
3,2022-01-01 00:00:01,2022-01-01,WIRTUALapp,HAPPY NEW YEAR 2022! Experience a new way to e...,2022-01-01 01:00:00,2022-01-01 00:01:00,2022-01-02 00:00:00+00:00,2022-01-01 00:30:00,2022-01-01 00:15:00,happy new year experience new way exercise wir...,0.513676
4,2022-01-01 00:00:01,2022-01-01,takeoff_tech,【$TKO token #1】\n\nTKO token will be a utility...,2022-01-01 01:00:00,2022-01-01 00:01:00,2022-01-02 00:00:00+00:00,2022-01-01 00:30:00,2022-01-01 00:15:00,tko token tko token utility token service fee ...,0.520613
...,...,...,...,...,...,...,...,...,...,...,...
1331663,2022-01-31 23:59:46,2022-01-31,deyonte_btc,One of the largest asset managers on the plane...,2022-02-01 00:00:00,2022-02-01 00:00:00,2022-02-01 00:00:00+00:00,2022-02-01 00:00:00,2022-02-01 00:00:00,one largest asset manager planet trillion aum ...,0.651603
1331664,2022-01-31 23:59:52,2022-01-31,Baripondiss,"Lol... Oh, you just realized, you're still ear...",2022-02-01 00:00:00,2022-02-01 00:00:00,2022-02-01 00:00:00+00:00,2022-02-01 00:00:00,2022-02-01 00:00:00,lol oh realized youre still early welcome bitcoin,0.765540
1331665,2022-01-31 23:59:52,2022-01-31,galaxy_orion,@julesapril8 @jimcramer I know. \n\n- He instr...,2022-02-01 00:00:00,2022-02-01 00:00:00,2022-02-01 00:00:00+00:00,2022-02-01 00:00:00,2022-02-01 00:00:00,know instruct agency ban bitcoin russia get wo...,0.042174
1331666,2022-01-31 23:59:55,2022-01-31,BitcoinFeesCash,Updated Bitcoin transaction fees: \n \nBCH ...,2022-02-01 00:00:00,2022-02-01 00:00:00,2022-02-01 00:00:00+00:00,2022-02-01 00:00:00,2022-02-01 00:00:00,updated bitcoin transaction fee bch next block...,0.655889


In [43]:
current_df.to_csv("Tweets_with_sentiment_elkulako.csv", index = False)

In [33]:
#Group the dataset by 15 min hour and 30 min
df_15m = current_df.groupby("Date_15m").agg({"datetime":list,"text":"\n".join,"username":"\n".join,"Positive_score":"mean"}).reset_index()
df_30m = current_df.groupby("Date_30m").agg({"datetime":list,"text":"\n".join,"username":"\n".join,"Positive_score":"mean"}).reset_index()
df_1H = current_df.groupby("Date_hourly").agg({"datetime":list,"text":"\n".join,"username":"\n".join,"Positive_score":"mean"}).reset_index()

In [34]:
sentiment_15m = df_15m

In [35]:
fin_15m = pd.read_csv("/content/sample_data/BTCUSDT_15m.csv")

In [36]:
fin_15m

,date,open,close,Return,Return_sign
0,2022-01-01 00:00:00,46216.93,46332.51,0.002501,1
1,2022-01-01 00:15:00,46332.52,46375.42,0.000926,1
2,2022-01-01 00:30:00,46375.42,46610.81,0.005076,1
3,2022-01-01 00:45:00,46610.81,46656.13,0.000972,1
4,2022-01-01 01:00:00,46656.14,46766.99,0.002376,1
...,...,...,...,...,...
2971,2022-01-31 22:45:00,38590.00,38412.46,-0.004601,0
2972,2022-01-31 23:00:00,38412.45,38451.36,0.001013,1
2973,2022-01-31 23:15:00,38451.36,38412.23,-0.001018,0
2974,2022-01-31 23:30:00,38412.23,38369.11,-0.001123,0


In [37]:
# Convert both columns to datetime format
sentiment_15m["Date_15m"] = pd.to_datetime(sentiment_15m["Date_15m"])
fin_15m["date"] = pd.to_datetime(fin_15m["date"])

# Now perform the merge
df_15m = pd.merge(sentiment_15m, fin_15m, left_on="Date_15m", right_on="date", how="inner")


In [39]:
df_15m
# Remove colums datetime	text	username
df_15m = df_15m.drop(columns=["datetime","text","username","date"])

In [40]:
df_15m

,Date_15m,Positive_score,open,close,Return,Return_sign
0,2022-01-01 00:15:00,0.639253,46332.52,46375.42,0.000926,1
1,2022-01-01 00:30:00,0.647286,46375.42,46610.81,0.005076,1
2,2022-01-01 00:45:00,0.662242,46610.81,46656.13,0.000972,1
3,2022-01-01 01:00:00,0.623321,46656.14,46766.99,0.002376,1
4,2022-01-01 01:15:00,0.649561,46766.99,46815.33,0.001034,1
...,...,...,...,...,...,...
2970,2022-01-31 22:45:00,0.649623,38590.00,38412.46,-0.004601,0
2971,2022-01-31 23:00:00,0.658045,38412.45,38451.36,0.001013,1
2972,2022-01-31 23:15:00,0.656948,38451.36,38412.23,-0.001018,0
2973,2022-01-31 23:30:00,0.646117,38412.23,38369.11,-0.001123,0


In [41]:
def create_model(num_layers=6, neurons_per_layer=48, input_shape=(10,), output_units=1, activation='relu'):
    model = Sequential()
    model.add(Dense(neurons_per_layer, activation=activation, input_shape=input_shape))

    # Agregar las capas ocultas especificadas
    for _ in range(num_layers - 1):
        model.add(Dense(neurons_per_layer, activation=activation))

    # Capa de salida
    model.add(Dense(output_units, activation='sigmoid'))  # Cambia 'sigmoid' a 'softmax' si tienes una clasificación multiclase

    # Compilar el modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

X  = np.array(df_15m['Positive_score']).reshape(-1, 1)
y  = np.array(df_15m['Return_sign'])
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Definir parámetros de la red
num_layers_list = [2, 3, 4,6,8]  # Diferentes números de capas a probar
neurons_per_layer_list = [16, 32, 64,128]  # Diferentes números de neuronas en cada capa

# Probar diferentes configuraciones de capas y neuronas
for num_layers in num_layers_list:
    for neurons_per_layer in neurons_per_layer_list:
        print(f"\nEntrenando modelo con {num_layers} capas ocultas y {neurons_per_layer} neuronas por capa")

        # Crear el modelo
        model = create_model(num_layers=num_layers, neurons_per_layer=neurons_per_layer, input_shape=(1,))

        # Definir early stopping para evitar el sobreajuste
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Entrenar el modelo
        history = model.fit(X_train, Y_train, validation_split=0.2, epochs=200, batch_size=16, callbacks=[early_stopping,TqdmCallback(verbose=1)], verbose=1)

        # Evaluar el modelo
        loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
        print(f"Modelo con {num_layers} capas y {neurons_per_layer} neuronas -> Pérdida: {loss:.4f}, Precisión: {accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Entrenando modelo con 2 capas ocultas y 16 neuronas por capa


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5035 - loss: 0.6951 - val_accuracy: 0.5021 - val_loss: 0.6937
Epoch 2/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4938 - loss: 0.6939 - val_accuracy: 0.5021 - val_loss: 0.6938
Epoch 3/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5223 - loss: 0.6922 - val_accuracy: 0.5021 - val_loss: 0.6933
Epoch 4/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5284 - loss: 0.6923 - val_accuracy: 0.5021 - val_loss: 0.6931
Epoch 5/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4962 - loss: 0.6933 - val_accuracy: 0.5021 - val_loss: 0.6931
Epoch 6/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5191 - loss: 0.6928 - val_accuracy: 0.5021 - val_loss: 0.6932
Epoch 7/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4765 - loss: 0.6937 - val_accuracy: 0.5021 - val_loss: 0.6938
Epoch 8/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5226 - loss: 0.6923 - val_accu

...

0batch [00:00, ?batch/s]

Epoch 1/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.5042 - loss: 0.6933 - val_accuracy: 0.5021 - val_loss: 0.6931
Epoch 2/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4918 - loss: 0.6934 - val_accuracy: 0.5021 - val_loss: 0.6934
Epoch 3/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5189 - loss: 0.6929 - val_accuracy: 0.5021 - val_loss: 0.6932
Epoch 4/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5299 - loss: 0.6926 - val_accuracy: 0.5021 - val_loss: 0.6932
Epoch 5/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5149 - loss: 0.6931 - val_accuracy: 0.5021 - val_loss: 0.6932
Epoch 6/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5104 - loss: 0.6931 - val_accuracy: 0.5021 - val_loss: 0.6934
Modelo con 8 capas y 64 neuronas -> Pérdida: 0.6929, Precisión: 0.5328

Entrenando modelo con 8 capas ocultas y 128 neuronas por capa


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.4618 - loss: 0.6943 - val_accuracy: 0.5021 - val_loss: 0.6933
Epoch 2/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5298 - loss: 0.6927 - val_accuracy: 0.5021 - val_loss: 0.6932
Epoch 3/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4584 - loss: 0.6937 - val_accuracy: 0.5021 - val_loss: 0.6933
Epoch 4/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5167 - loss: 0.6928 - val_accuracy: 0.5021 - val_loss: 0.6932
Epoch 5/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5175 - loss: 0.6931 - val_accuracy: 0.5021 - val_loss: 0.6934
Epoch 6/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5129 - loss: 0.6930 - val_accuracy: 0.5021 - val_loss: 0.6933
Epoch 7/200
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5075 - loss: 0.6931 - val_accuracy: 0.5021 - val_loss: 0.6934
Modelo con 8 capas y 128 neuronas -> Pérdida: 0.6925, Precisión: 0.5328


In [49]:
twitter_15min

,datetime,aggregated_positive_score
0,2022-01-01 00:00:00,0.639253
1,2022-01-01 00:15:00,0.647286
2,2022-01-01 00:30:00,0.662242
3,2022-01-01 00:45:00,0.623321
4,2022-01-01 01:00:00,0.649561
...,...,...
2971,2022-01-31 22:45:00,0.658045
2972,2022-01-31 23:00:00,0.656948
2973,2022-01-31 23:15:00,0.646117
2974,2022-01-31 23:30:00,0.656399


In [ ]:
# Asegúrate de que las columnas de fecha estén en formato datetime
current_df['datetime'] = pd.to_datetime(current_df['datetime'])
fin_15m['date'] = pd.to_datetime(fin_15m['date'])
# Configura el índice para resampling basado en datetime
current_df.set_index('datetime', inplace=True)
